<a href="https://colab.research.google.com/github/screemix/The_Glorious_Chargers/blob/main/Shipment_features_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd

df = pd.read_csv('/shipments2020-01-01.csv')
for name in ['/shipments2020-03-01.csv', '/shipments2020-04-30.csv', '/shipments2020-06-29.csv']:
  df_tmp =  pd.read_csv(name)
  frames = [df, df_tmp]
  df = pd.concat(frames)
df.head(100)

,user_id,ship_address_id,shipment_id,order_id,order_created_at,order_completed_at,shipment_starts_at,retailer,s.order_state,shipment_state,s.city_name,s.store_id,total_cost,rate,dw_kind,promo_total,total_weight,platform,os,shipped_at
0,11019,171906,322307,2156687,2017-08-03 21:25:23,2020-02-18 14:07:00,2020-02-20 07:00:00,METRO,complete,shipped,Москва,21.0,168.0,0.0,courier,0.0,30170.0,web,windows,2020-02-20 08:08:54
1,62278,278832,387023,3021953,2018-03-02 17:22:04,2020-01-03 13:09:26,2020-01-03 17:00:00,METRO,complete,shipped,Москва,10.0,98.0,0.0,courier,-150.0,11305.0,web,windows,2020-01-03 18:10:40
2,905126,468407,388943,3018198,2018-02-28 11:32:47,2020-02-12 12:39:28,2020-02-13 11:00:00,METRO,complete,shipped,Москва,21.0,98.0,5.0,courier,0.0,13589.0,app,ios,2020-02-13 12:33:53
3,21412,61962,421048,3030227,2018-03-07 20:37:27,2020-01-25 11:58:56,2020-01-25 18:00:00,METRO,complete,shipped,Москва,8.0,158.0,0.0,courier,0.0,9726.0,web,mac,2020-01-25 19:55:32
4,42110,378297,442659,2923996,2017-12-24 11:19:04,2020-01-07 14:30:44,2020-01-07 19:00:00,METRO,complete,shipped,Москва,2.0,163.0,0.0,courier,0.0,30323.0,web,windows,2020-01-07 19:51:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,167421,1067271,858662,3861524,2019-03-14 11:08:50,2020-01-29 09:54:43,2020-01-29 16:00:00,METRO,complete,shipped,Москва,8.0,0.0,0.0,courier,0.0,13634.0,web,windows,2020-01-29 16:43:02
96,147256,1069258,859942,3863487,2019-03-14 21:10:25,2020-02-06 12:06:08,2020-02-06 16:00:00,METRO,complete,shipped,Москва,68.0,158.0,0.0,courier,0.0,3814.0,app,ios,2020-02-06 16:51:27
97,161717,1023057,861815,3815741,2019-02-28 20:11:42,2020-01-28 14:32:44,2020-02-03 05:00:00,METRO,canceled,canceled,Москва,14.0,0.0,0.0,courier,0.0,57485.0,app,ios,NaN
98,130777,1073461,862632,3867605,2019-03-16 10:56:24,2020-02-08 13:07:18,2020-02-09 05:00:00,METRO,complete,shipped,Москва,1.0,158.0,0.0,courier,0.0,8745.0,web,windows,2020-02-09 05:45:48


In [19]:
# maximal cost
df_max_total_cost = df[['user_id','total_cost']].groupby("user_id").max()

# mean cost
df_mean_total_cost = df[['user_id','total_cost']].groupby("user_id").mean()

#overall number of orders
df_orders = df[['user_id', 'order_id']].groupby("user_id").count().rename(columns={'order_id': 'orders_overall'})

#not_canceled/overall ratio
is_canceled = df['s.order_state']!='canceled'

df_not_canceled_orders = df[is_canceled][['user_id', 'order_id']].groupby("user_id").count().rename(columns={'order_id': 'not_canceled_id'})
frames = [df_not_canceled_orders, df_orders]
df_orders_ratio = pd.concat(frames, axis=1, join='outer')
df_orders_ratio["not_calceled_overall_ratio"] = df_orders_ratio["not_canceled_id"] / df_orders_ratio["orders_overall"]
df_orders_ratio = df_orders_ratio.drop(['orders_overall', 'not_canceled_id'], axis=1)

frames = [df_orders_ratio, df_orders, df_mean_total_cost]


df_all = pd.concat(frames, axis=1, join='outer')
df_all.head()


,not_calceled_overall_ratio,orders_overall,total_cost
user_id,,,
1400,NaN,1,158.000000
1459,0.043478,23,151.695652
1540,1.000000,1,158.000000
1541,1.000000,1,163.000000
1577,1.000000,4,157.000000


In [20]:
#mean weigth
print(df.head())
df_mean_total_weight = df[['user_id','total_weight']].groupby("user_id").mean()

# mean of rates > 0
df_rate = df[['user_id','rate']]
df_rate = df_rate[df_rate["rate"]>0].groupby("user_id").mean()
df_rate.shape

   user_id  ship_address_id  shipment_id  ...  platform       os           shipped_at
0    11019           171906       322307  ...       web  windows  2020-02-20 08:08:54
1    62278           278832       387023  ...       web  windows  2020-01-03 18:10:40
2   905126           468407       388943  ...       app      ios  2020-02-13 12:33:53
3    21412            61962       421048  ...       web      mac  2020-01-25 19:55:32
4    42110           378297       442659  ...       web  windows  2020-01-07 19:51:37

[5 rows x 20 columns]


(339923, 1)

In [21]:
#merging with table to get phone_id matching for each user
df = pd.read_csv('/phone_user_ship_mapping.csv')
print(df.head())
new_df = df.join(df_all, on='user_id')
new_df = new_df.drop('ship_address_id', axis=1)
print(new_df.head())
new_df.to_csv('new_features.csv',index=False)

   user_id  ship_address_id  phone_id
0     4708             8531         2
1     6557            17410         3
2     7729            23364         4
3    53084            45029         6
4    16408            49164         8
   user_id  phone_id  not_calceled_overall_ratio  orders_overall  total_cost
0     4708         2                    1.000000             5.0  189.000000
1     6557         3                    1.000000             4.0   42.000000
2     7729         4                    0.857143             7.0  136.428571
3    53084         6                    1.000000             9.0  178.666667
4    16408         8                    1.000000             2.0  158.000000


In [22]:
#import numpy as np

# most frequently used os for user
#df_os = df[['user_id','os']].groupby("user_id").agg(list)
#pridf_os.head)
